In [17]:
import numpy as np
import pandas as pd
import torch

import urllib
from PIL import Image
from torchvision import transforms

from os import listdir
from os.path import isfile, join

In [16]:
#References https://pytorch.org/hub/pytorch_vision_alexnet/

model = torch.hub.load('pytorch/vision:v0.9.0', 'alexnet', pretrained=True)
model.eval()

Using cache found in /Users/William/.cache/torch/hub/pytorch_vision_v0.9.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [39]:
# url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
# try: urllib.URLopener().retrieve(url, filename)
# except: urllib.request.urlretrieve(url, filename)

mypath = "dog/Images/n02110958-pug/"
files = [f for f in listdir(mypath) if (isfile(join(mypath, f)))]
    
probs = []

for filename in files:
    input_image = Image.open(mypath + filename)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    #print(output[0])
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    p = torch.nn.functional.softmax(output[0], dim=0)
    probs.append(p)
    #print(p)

In [40]:
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
for j in range(len(files)):
    top5_prob, top5_catid = torch.topk(probs[j], 5)
    print(files[j])
    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())
    print("-------------------------------------------------------")

n02110958_12350.jpg
pug 0.9991470575332642
French bulldog 0.000468804850243032
bull mastiff 0.00031585985561832786
Brabancon griffon 2.0155748643446714e-05
Norwegian elkhound 1.1358810297679156e-05
-------------------------------------------------------
n02110958_12807.jpg
Brabancon griffon 0.30073127150535583
pug 0.2831593155860901
bull mastiff 0.2024223655462265
malinois 0.09034732729196548
boxer 0.036915916949510574
-------------------------------------------------------
n02110958_16334.jpg
pug 0.4855116307735443
bull mastiff 0.47809481620788574
boxer 0.014456638135015965
Great Dane 0.007367873564362526
sulphur-crested cockatoo 0.002461828291416168
-------------------------------------------------------
n02110958_15615.jpg
pug 0.769454836845398
bull mastiff 0.07175210118293762
French bulldog 0.05530662462115288
Labrador retriever 0.02567906305193901
whippet 0.01997901126742363
-------------------------------------------------------
n02110958_11261.jpg
pug 0.9875671863555908
French b

pug 0.5891711115837097
French bulldog 0.045522771775722504
Newfoundland 0.03960627689957619
Scotch terrier 0.038589492440223694
giant schnauzer 0.03854414075613022
-------------------------------------------------------
n02110958_15527.jpg
pug 0.7572988867759705
Boston bull 0.10611902922391891
French bulldog 0.0580592155456543
bull mastiff 0.02785111404955387
boxer 0.015901202335953712
-------------------------------------------------------
n02110958_12260.jpg
pug 0.9918498992919922
French bulldog 0.005934949032962322
bath towel 0.0007784810150042176
bull mastiff 0.00037827197229489684
Brabancon griffon 0.0002661166072357446
-------------------------------------------------------
n02110958_12275.jpg
pug 0.9947861433029175
French bulldog 0.004837148357182741
bull mastiff 0.00019804862677119672
Boston bull 0.00014699067105539143
Staffordshire bullterrier 1.043140855472302e-05
-------------------------------------------------------
n02110958_7255.jpg
pug 0.6681349277496338
bull mastiff 0.